# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv
import joblib, pickle

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

#from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-project'

experiment=Experiment(ws, experiment_name)

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_cluster = 'automl-cluster'

try:
    comp_trget = ComputeTarget(workspace=ws, name=compute_cluster)
    print('Cluster already exists')
except ComputeTargetException:
    compute_confg = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",max_nodes=4)
    comp_trget = ComputeTarget.create(ws,compute_cluster,compute_confg)

In [4]:
data_link = 'https://raw.githubusercontent.com/murphy999/Udacity-Azure-ML-Scholarship/master/nd00333-capstone/starter_file/Telco-Customer-Churn.csv'
input_data = TabularDatasetFactory.from_delimited_files(path=data_link)

In [5]:
from model_training import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(input_data)
predict_col = pd.DataFrame(y)
predict_col.head()
complete_data = pd.concat([x,predict_col],axis=1)
complete_data.head()

/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook131106/code/Users/odl_user_131106/model_training.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Partner'] = df.Partner.apply(lambda s: 1 if s == True else 0)
/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook131106/code/Users/odl_user_131106/model_training.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Dependents'] = df.Dependents.apply(lambda s: 1 if s == True else 0)
/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook1311

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,...,gender_Male,InternetService_Fiber optic,InternetService_No,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Contract_One year,Contract_Two year,Churn
0,0,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,34,1,0,1,0,1,0,...,1,0,0,0,0,0,1,1,0,0
2,0,0,0,2,1,0,1,1,0,0,...,1,0,0,0,0,0,1,0,0,1
3,0,0,0,45,0,0,1,0,1,1,...,1,0,0,1,0,0,0,1,0,0
4,0,0,0,2,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1


In [6]:
# split data into training and test data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(complete_data,test_size=0.25)
train_data.head()

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,...,gender_Male,InternetService_Fiber optic,InternetService_No,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Contract_One year,Contract_Two year,Churn
4130,0,0,0,10,0,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0
1680,0,1,1,56,1,1,0,0,0,0,...,1,0,1,0,0,0,1,1,0,0
3541,0,1,0,49,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
5550,1,0,0,21,1,0,0,0,0,0,...,1,0,1,1,0,0,0,1,0,0
5313,0,1,0,56,1,0,0,0,0,0,...,1,1,0,0,0,0,1,0,0,0


In [7]:
# Save training data to a csv file 
train_data.to_csv("customer_churn_traindata.csv")

In [17]:
data_store = ws.get_default_datastore()
data_store.upload(src_dir='./',target_path='./')
ftrain_data = Dataset.Tabular.from_delimited_files(path = [(data_store, './customer_churn_traindata.csv')])

Uploading an estimated of 6 files
Target already exists. Skipping upload for automl.ipynb
Target already exists. Skipping upload for customer_churn_traindata.csv
Target already exists. Skipping upload for model_training.py
Target already exists. Skipping upload for score.py
Target already exists. Skipping upload for __pycache__/model_training.cpython-36.pyc
Uploading ./.ipynb_checkpoints/automl-checkpoint.ipynb
Uploaded ./.ipynb_checkpoints/automl-checkpoint.ipynb, 1 files out of an estimated total of 2
Uploaded 1 files


In [36]:
temp = ftrain_data.drop_columns('Column1')

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [37]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 25,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
                             compute_target=comp_trget,
                             task = "classification",
                             training_data=temp,
                             label_column_name="Churn",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             n_cross_validations=7,
                             **automl_settings
)

In [38]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

Running on remote.
No run_configuration provided, running on automl-cluster with default configuration
Running on remote compute: automl-cluster
Parent Run ID: AutoML_3f6b8572-d27d-43d6-8158-54dff8e7a1b0

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing val

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [39]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_3f6b8572-d27d-43d6-8158-54dff8e7a1b0',
 'target': 'automl-cluster',
 'status': 'Completed',
 'startTimeUtc': '2020-12-20T10:52:00.559555Z',
 'endTimeUtc': '2020-12-20T11:18:26.905162Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '7',
  'target': 'automl-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-project","subscription_id":"6971f5ac-8af1-446e-8034-05acea24681f","resource_group":"aml-quickstarts-131106","workspace_name":"quick-starts-ws-131106","region":"southcentralus","compute_target":"automl-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":7,"y_min":null,"y_max":null,"num_clas

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [40]:
best_automl, best_fit_model = remote_run.get_output()
print(best_fit_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                        max_iter=-1,
                                                                                        probability=True,
                                                                                        random_state=None,
                                                                              

In [41]:
#TODO: Save the best model
joblib.dump(best_fit_model, 'best_fit_automl_model.pkl')

['best_fit_automl_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [42]:
# Register the Model
from azureml.core.model import Model
model = Model.register(workspace = ws, model_name = 'best_fit_automl_model', model_path = 'best_fit_automl_model.pkl')
print(model.name, model.id, model.version, sep='\t')

Registering model best_fit_automl_model
best_fit_automl_model	best_fit_automl_model:1	1


In [49]:
# Deploy the Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, Webservice, AciWebservice
from azureml.core.conda_dependencies import CondaDependencies
import azureml.train.automl

# Create the environment
env = best_automl.get_environment()
conda_dep = CondaDependencies()

# Define the packages needed by the model and the script
#conda_dep.add_conda_package("numpy")
#conda_dep.add_conda_package("scikit-learn")
#conda_dep.add_conda_package("pandas")
# Define azureml-defaults as a pip dependency
#conda_dep.add_pip_package("azureml-defaults")
#conda_dep.add_pip_package("azureml-train")

# Adds dependencies to PythonSection of myenv
#env.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script='score.py', environment=env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True)
service = Model.deploy(ws, "customerservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..............
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://a3812d19-c07d-4375-987d-0f4e7f480fe1.southcentralus.azurecontainer.io/score
http://a3812d19-c07d-4375-987d-0f4e7f480fe1.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [54]:
import requests
import json

# URL for the web service
scoring_uri = 'http://a3812d19-c07d-4375-987d-0f4e7f480fe1.southcentralus.azurecontainer.io/score'

# Set the content type
headers = {'Content-Type': 'application/json'}

# Two sets of data to score, so we get two results back
data = {"data":
        [{
                "gender_Male" :0,
                "SeniorCitizen":0,
                "Dependents" :0,
                "Partner" :0,
                "tenure":2,
                "PhoneService" :1,
                "MultipleLines" :1,
                "InternetService_Fiber optic" :1,
                "InternetService_No":0,
                "OnlineSecurity" :0,
                "OnlineBackup" :0,
                "DeviceProtection" :0,
                "TechSupport" :0,
                "StreamingTV" :0,
                "StreamingMovies" :1,
                "Contract_One year" : 0,
                "Contract_Two year":0,
                "PaperlessBilling" :1,
                "PaymentMethod_Bank transfer (automatic)":0,
                "PaymentMethod_Credit card (automatic)":0,
                "PaymentMethod_Mailed check":0,
                "PaymentMethod_Electronic check" :1,
                "MonthlyCharges" : 85.7,
                "TotalCharges" : 169.8 }
        ]
        }
# Convert to JSON string
input_data = json.dumps(data)

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)

print("Response Code : ", resp.status_code)
print("Predicted Value : ",resp.text)

Response Code :  200
Predicted Value :  [1]


TODO: In the cell below, print the logs of the web service and delete the service

In [55]:
# Web Service Logs
print(service.get_logs())

2020-12-20T11:42:31,274427400+00:00 - rsyslog/run 
2020-12-20T11:42:31,269551100+00:00 - iot-server/run 
2020-12-20T11:42:31,298459900+00:00 - gunicorn/run 
2020-12-20T11:42:31,298472400+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_e1251fbb90896de9de2cde1255eba7bb/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_e1251fbb90896de9de2cde1255eba7bb/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_e1251fbb90896de9de2cde1255eba7bb/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_e1251fbb90896de9de2cde1255eba7bb/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_e1251fbb90896de9de2cde1255eba7bb/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [58]:
# Delete the service
service.delete()

No service with name customerservice found to delete.
